## Waypoint Analysis with CargoTimeSeries endpoint

This notebook will give a tutorial on how to query our waypoint traffic data through pythonSDK.

In this notebook, we will query data for <b> Cape of Good Hope (COGH) and Bab-el-Mandeb</b> as our example.

## Import Libraries

In [1]:
import vortexasdk as v
import pandas as pd
from datetime import datetime
import plotly.express as px
import numpy as np

c:\Users\OuiWeinJien\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


## Extract relevant id

In [3]:
eos = [p.id for p in v.Geographies().search('East of Suez').to_list() if p.layer==['alternative_region']]
wos = [p.id for p in v.Geographies().search('West of Suez').to_list() if p.layer==['alternative_region']]
red_sea = [p.id for p in v.Geographies().search('Red Sea').to_list() if p.layer==['shipping_region_v2']]
bab_el_mandeb= [p.id for p in v.Geographies().search('bab-el-mandeb').to_list() if p.layer==['waypoint']]
cogh=[p.id for p in v.Geographies().search('Cape of Good Hope').to_list() if p.name=='Cape Of Good Hope']

russian_grades=['c8803c073c2980d9849784c16bcd41d7f7b7561d2c8c1799a16166527b120242', 
                '04c4ea46fc6fccfe8388717f399b40e5e348f9a98c451d99e7366caa998191a7', 
                'a6c0a31f361b480bddfb93b1b93f38f4989013ae54b2b147ab18116fdfbebdb7',
                '7cab5376bbb441feabdafc7b9f6272c2f9cbd10deb604bed8fa7f9986b2a39a9',
                '20c87d7477f8cfe103904d9af911668c9a69bd03690e74e52e4386ad34e054ee', 
                'fef83ff1194c77d969693ee17c08e1112878e184e3449400e5904bc47b0119fe',
                '0fff12441aa3bf28aa568134ef527d09f741c543a49c0db8c03a195a0bbc481e', 
                '4ff810b966104a658d22e7155596c11bedcd1e3e81594e6187ab88ec816d95d6', 
                '0c703e0e14d99572c62f15a26fdf2c9f0c00e111e625cd40b8b337154e81d46d',
                '1dfa78c965d56ab9d7b953e424d9160e9f79d0275e460c74994193a328f51537', 
                '054b79baa9a54171be2582c9b4f8fa82d0386d41c1914700a498dc5b95e8daba', 
                '74a8bc823d4742318d38072f81068fac1ac02ac66e0d491ea4366007bb95c602']

sb_origins=red_sea+wos

cpp = [p.id for p in v.Products().search('Clean Petroleum Products').to_list() if p.name=='Clean Petroleum Products']
dpp = [p.id for p in v.Products().search('Dirty Petroleum Products').to_list() if p.name=='Dirty Petroleum Products']
crude = [p.id for p in v.Products().search('Crude/Condensates').to_list() if p.name=='Crude/Condensates']
lng = [p.id for p in v.Products().search('LNG').to_list() if p.name=='LNG']
lpg = [p.id for p in v.Products().search('LPG+').to_list() if p.name=='LPG+']

other_cpp = [p.id for p in v.Products().search('Other Clean Products').to_list() if p.name=='Other Clean Products']
cond = [p.id for p in v.Products().search('Condensates').to_list() if p.name=='Condensates']
biodiesel = [p.id for p in v.Products().search('Biodiesel').to_list() if p.name=='Biodiesel']

assert len(other_cpp)==1
assert len(cond)==1
assert len(biodiesel)==1

2024-09-17 13:43:07,775 vortexasdk.client — WARNING — You are using vortexasdk version 0.73.0, however version 0.73.1 is available.
You should consider upgrading via the 'pip install vortexasdk --upgrade' command.


## Helper function

In [10]:
def plotting_data(df1,df2, moving_average = None):
    df = df1.merge(df2, on = 'key', how = 'inner', suffixes=('_bem', '_cogh'))
    if moving_average:
        df['value_bem'] = df['value_bem'].rolling(window=moving_average).mean()
        df['value_cogh'] = df['value_cogh'].rolling(window=moving_average).mean()
    fig = px.line(df[df['key']>'2023-01-01'], x='key', y=['value_bem', 'value_cogh'], title='Bab el Mandeb vs Cape of Good Hope', 
                  labels={'value_bem':'Bab el Mandeb', 'value_cogh':'Cape of Good Hope'})
    fig.show()

## Analysis

### Crude/DPP northbound (East to West)

In [6]:
df1 = v.CargoTimeSeries().search(
    filter_activity = 'waypoint_start',
    filter_time_min = datetime(2023,1,1),
    filter_time_max = datetime.today(),
    filter_vessel_classes = ['oil_aframax', 'oil_suezmax', 'oil_vlcc'],
    filter_products = crude+dpp,
    filter_origins = eos,
    exclude_origins = red_sea,
    filter_waypoints = bab_el_mandeb,
    timeseries_frequency = 'day',
    timeseries_unit = 't',
    ).to_df()

df2 = v.CargoTimeSeries().search(
    filter_activity = 'waypoint_start',
    filter_time_min = datetime(2023,1,1),
    filter_time_max = datetime.today(),
    filter_vessel_classes = ['oil_aframax', 'oil_suezmax', 'oil_vlcc'],
    filter_products = crude+dpp,
    filter_origins = eos,
    exclude_origins = red_sea,
    filter_waypoints = cogh,
    timeseries_frequency = 'day',
    timeseries_unit = 't',
    ).to_df()

In [11]:
plotting_data(df1,df2, moving_average=14)

### Crude/DPP Southbound (West to East)

In [12]:
df3 = v.CargoTimeSeries().search(
    filter_activity = 'waypoint_start',
    filter_time_min = datetime(2023,1,1),
    filter_time_max = datetime.today(),
    filter_vessel_classes = ['oil_aframax', 'oil_suezmax', 'oil_vlcc'],
    filter_products = crude+dpp,
    filter_origins = sb_origins,
    filter_waypoints = bab_el_mandeb,
    timeseries_frequency = 'day',
    timeseries_unit = 't',
    ).to_df()

df4 = v.CargoTimeSeries().search(
    filter_activity = 'waypoint_start',
    filter_time_min = datetime(2023,1,1),
    filter_time_max = datetime.today(),
    filter_vessel_classes = ['oil_aframax', 'oil_suezmax', 'oil_vlcc'],
    filter_products = crude+dpp,
    filter_origins =sb_origins,
    filter_waypoints = cogh,
    timeseries_frequency = 'day',
    timeseries_unit = 't',
    ).to_df()

In [13]:
plotting_data(df3,df4, moving_average=14)

### CPP Northbound (East to West)

In [14]:
df5 = v.CargoTimeSeries().search(
    filter_activity = 'waypoint_start',
    filter_time_min = datetime(2023,1,1),
    filter_time_max = datetime.today(),
    exclude_vessel_classes = ['oil_specialised', 'oil_coastal'],
    filter_products = cpp,
    exclude_products = lpg,
    filter_origins = eos,
    exclude_origins = red_sea,
    filter_waypoints = bab_el_mandeb,
    timeseries_frequency = 'day',
    timeseries_unit = 't',
    ).to_df()

df6 = v.CargoTimeSeries().search(
    filter_activity = 'waypoint_start',
    filter_time_min = datetime(2023,1,1),
    filter_time_max = datetime.today(),
    exclude_vessel_classes = ['oil_specialised', 'oil_coastal'],
    filter_products = cpp,
    exclude_products = lpg,
    filter_origins = eos,
    exclude_origins = red_sea,
    filter_waypoints = cogh,
    timeseries_frequency = 'day',
    timeseries_unit = 't',
    ).to_df()

In [15]:
plotting_data(df5,df6,moving_average=14)

### CPP Southbound (West to East)

In [16]:
df7 = v.CargoTimeSeries().search(
    filter_activity = 'waypoint_start',
    filter_time_min = datetime(2023,1,1),
    filter_time_max = datetime.today(),
    exclude_vessel_classes = ['oil_specialised', 'oil_coastal'],
    filter_products = cpp,
    exclude_products = lpg,
    filter_origins = sb_origins,
    filter_waypoints = bab_el_mandeb,
    timeseries_frequency = 'day',
    timeseries_unit = 't',
    ).to_df()

df8 = v.CargoTimeSeries().search(
    filter_activity = 'waypoint_start',
    filter_time_min = datetime(2023,1,1),
    filter_time_max = datetime.today(),
    exclude_vessel_classes = ['oil_specialised', 'oil_coastal'],
    filter_products = cpp,
    exclude_products = lpg,
    filter_origins = sb_origins,
    filter_waypoints = cogh,
    timeseries_frequency = 'day',
    timeseries_unit = 't',
    ).to_df()

In [17]:
plotting_data(df7,df8,moving_average=14)

## Conclusion

In this notebook, it has demonstrated how to perform waypoint analysis (cargo quantity that enter the waypoint daily) using CargoTimeSeries endpoint.

From the charts above, you can see that the market dynamic changes significantly after Jan 2024, reason being that the market player is trying to avoid risk transiting through Bab-el-Mandeb area since red sea incident.